In [1]:
import gmaps
import pandas as pd
import requests
import json

# Google developer API key
from api_keys import g_key
from pprint import pprint

# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
cw_df = pd.read_csv('output_data/cities.csv')
cw_df.head()

,City,Longitude,Latitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date Time
0,dikson,80.5464,73.5069,-4.56,92,100,18.63,RU,1643662122
1,mys shmidta,26.8910,-33.5906,75.60,94,100,7.78,ZA,1643661943
2,port alfred,13.2343,-8.8368,78.80,78,0,9.22,AO,1643662014
3,luanda,-73.9662,42.6001,29.25,41,9,1.01,US,1643662704
4,albany,-68.5145,63.7506,-27.31,74,20,1.99,CA,1643662408


In [3]:
cw_df.sort_values('Humidity', ascending=False)

,City,Longitude,Latitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date Time
289,klaksvik,102.2797,60.3400,-6.90,100,94,1.81,RU,1643663131
323,vanavara,112.4000,66.4167,-35.00,100,84,3.44,RU,1643663199
21,sorvag,-56.1549,72.7868,-19.95,100,87,2.84,GL,1643662142
268,karaul,125.2167,49.1833,-18.76,100,77,1.97,CN,1643663065
621,lagunas,129.5230,49.7580,-18.29,100,21,4.41,RU,1643663684
...,...,...,...,...,...,...,...,...,...
503,turukhansk,-2.8000,8.0333,76.66,12,98,2.51,CI,1643663531
467,beloha,1.0114,20.1986,68.52,12,33,10.63,ML,1643663317
498,mizdah,-11.0000,14.0000,87.57,10,68,7.07,ML,1643663530
505,mahibadhoo,-116.0542,34.1356,66.47,10,100,0.98,US,1643663532


In [4]:
locs = cw_df[["Latitude", "Longitude"]]
humidity = cw_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locs, weights=humidity, 
                                 dissipating=False,
                                 point_radius=5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
good_df = cw_df.loc[(cw_df['Max Temperature'] > 70) & (cw_df['Max Temperature'] < 80) \
                                          & (cw_df['Wind Speed'] < 10) \
                                          & (cw_df['Cloudiness'] < 10) \
                                          & (cw_df['Humidity'] < 60)]
good_df

,City,Longitude,Latitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date Time
101,las cruces,-109.9124,22.8909,79.52,56,1,1.99,MX,1643662703
112,grand gaube,-94.3986,35.3859,70.88,32,0,1.99,US,1643662846
288,omboue,75.4333,18.0833,70.61,23,3,7.58,IN,1643663131
296,sambava,6.2642,4.9247,74.79,41,7,4.27,NG,1643663132
510,paez,-90.4000,20.8667,78.21,52,1,8.32,MX,1643663533
590,pavlivka,-117.2711,32.9912,72.57,59,0,1.99,US,1643663675


In [11]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location" : None,
    "radius" : 5000,
    "keyword" : "hotel",
    "key" : g_key
}


In [12]:
coords = []
hotel_info = []
for index, row in good_df.iterrows():
    params['location'] = f"{row['Latitude']},{row['Longitude']}"
    hotel = requests.get(base_url, params=params).json()
    try:
        hotel_info.append({'name' : hotel['results'][0]['name'], 'city' : row['City'].title(), 'country' : row['Country']})
        coords.append((hotel['results'][0]['geometry']['location']['lat'],hotel['results'][0]['geometry']['location']['lng']))
    except:
        print(f"Missing field/result for {row['City'].title()}... skipping.")

In [13]:
hotel_info

[{'name': 'Medano Hotel and Suites', 'city': 'Las Cruces', 'country': 'MX'},
 {'name': 'Best Western Aspen Hotel', 'city': 'Grand Gaube', 'country': 'US'},
 {'name': 'Hotel Dhanashree', 'city': 'Omboue', 'country': 'IN'},
 {'name': 'The Monalisa Hotel and Suites', 'city': 'Sambava', 'country': 'NG'},
 {'name': 'Hotel Gutierrez', 'city': 'Paez', 'country': 'MX'},
 {'name': 'Holiday Inn Express & Suites Solana Beach-del Mar, an IHG Hotel',
  'city': 'Pavlivka',
  'country': 'US'}]

In [14]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
info_box = [info_box_template.format(**hotel) for hotel in hotel_info]
markers = gmaps.marker_layer(coords, info_box_content=info_box)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))